In [1]:
import ada
from ada.fem.meshing import GmshOptions
from ada.materials.metals import CarbonSteel

In [2]:
def make_fem(geom_repr) -> ada.Assembly:
    bm = ada.Beam("MyBeam", (0, 0.5, 0.5), (5, 0.5, 0.5), "IPE400", ada.Material("S420", CarbonSteel("S420")))
    assembly = ada.Assembly("MyAssembly") / [ada.Part("MyPart") / bm]
    part = bm.parent
    part.fem = bm.to_fem_obj(0.1, geom_repr, options=GmshOptions(Mesh_ElementOrder=1))
    nodes = bm.bbox.sides.back(return_fem_nodes=True)
    assembly.fem.add_bc(ada.fem.Bc("Fixed", ada.fem.FemSet("bc_nodes", nodes), [1, 2, 3, 4, 5, 6]))
    assembly.fem.add_step(ada.fem.StepEigen("Eigen", num_eigen_modes=10))
    return assembly

In [3]:
a = make_fem("shell")
ccx_res = a.to_fem("Cantilever_CCX_EIG_sh", "calculix", overwrite=True, execute=True)

Removing old files before copying new
Created a Calculix input deck at "c:\AibelProgs\ADA\scratch\Cantilever_CCX_EIG_sh"
--------------------------------------------------------------------------------
Starting Calculix simulation "Cantilever_CCX_EIG_sh" (on Windows) using 2 cpus
Finished Calculix simulation "Cantilever_CCX_EIG_sh"
--------------------------------------------------------------------------------


In [4]:
a = make_fem("shell")
ca_res = a.to_fem("Cantilever_CA_EIG_sh", "code_aster", overwrite=True, execute=True)

Removing old files before copying new
Created a Code_Aster input deck at "c:\AibelProgs\ADA\scratch\Cantilever_CA_EIG_sh"
--------------------------------------------------------------------------------
Starting CodeAster simulation "Cantilever_CA_EIG_sh" (on Windows) using 2 cpus
Finished CodeAster simulation "Cantilever_CA_EIG_sh"
--------------------------------------------------------------------------------


In [5]:
scene = ccx_res.to_trimesh(1, "DISP", "DISP", 1, 20)
scene.show()

In [11]:
scene = ca_res.to_trimesh(
    5.054232624598001, "modes___DEPL[0] - 5.05423", "modes___DEPL[0] - 5.05423", 5.054232624598001, 1
)
scene.show()

In [7]:
for cx in ccx_res.iter_results_by_field_value():
    print(cx)

NodalFieldData(name='DISP', step=1, components=['D1', 'D2', 'D3', 'ALL'], values=array([[ 7.21000e+02,  0.00000e+00,  0.00000e+00, -3.41087e-06],
       [ 7.23000e+02,  0.00000e+00, -0.00000e+00,  3.41087e-06],
       [ 7.24000e+02, -2.74524e-03,  1.10826e-01, -8.56491e-06],
       ...,
       [ 3.99900e+03, -9.31715e-04,  2.00099e-02, -7.00671e-05],
       [ 4.00000e+03, -1.37202e-03,  1.00144e-01,  5.60019e-06],
       [ 4.00200e+03, -1.37197e-03,  1.00145e-01,  5.58137e-06]]))
NodalFieldData(name='STRESS', step=1, components=['SXX', 'SYY', 'SZZ', 'SXY', 'SYZ', 'SZX'], values=array([[ 7.21000e+02, -2.03936e+08, -1.21381e+07, ...,  2.56137e+07,
        -2.82903e+06,  4.18204e+06],
       [ 7.23000e+02, -2.17485e+08, -1.88964e+07, ...,  2.14314e+07,
         9.66455e+05,  3.82411e+06],
       [ 7.24000e+02, -6.35956e+03, -2.79858e+04, ...,  3.27970e+04,
        -4.91189e+03,  2.93014e+03],
       ...,
       [ 3.99900e+03, -7.74873e+07,  1.20800e+04, ...,  2.10305e+06,
         1.04429

In [8]:
for ca in ca_res.iter_results_by_field_value():
    print(ca)

NodalFieldData(name='modes___DEPL[0] - 5.05423', step=5.054232624598001, components=['DX', 'DY', 'DZ', 'DRX', 'DRY', 'DRZ'], values=array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.00000000e+00, -2.47193551e-02,  9.98178713e-01, ...,
        -1.33139810e-02,  1.31013175e-04, -1.59825780e-19],
       [ 3.00000000e+00,  2.76803172e-15,  9.98178844e-01, ...,
        -1.34689994e-02, -3.07917368e-14,  2.74395442e-01],
       ...,
       [ 7.18000000e+02, -1.23347475e-02,  8.47167798e-01, ...,
         1.23743173e-02, -9.86470873e-05, -2.10067683e-20],
       [ 7.19000000e+02, -8.39569590e-03,  1.80613230e-01, ...,
         7.17880052e-03, -4.55272926e-05,  2.24459492e-20],
       [ 7.20000000e+02, -1.23526873e-02,  9.02048117e-01, ...,
         1.27844112e-02, -8.56076939e-05,  8.81627408e-20]]))
NodalFieldData(name='modes___DEPL[1] - 9.75129', step=9.75129233796498, components=['DX', 'DY', 'DZ', 'DRX', '